# **<center>avg_temperature</center>**

![](https://pic2.zhimg.com/v2-90d0508bebd533f277bdaa874bf1c15b_1440w.jpg)

---
## TODO List
- [ ] [avg_temperature](https://github.com/arthurtibame/iii-pypsark/blob/master/Spark-01.ipynb)
- [ ] avg_temperature2
- [ ] avg_temperature3

In [1]:
from pyspark import SparkContext, SparkConf

## STEP1. 
    從hdfs 撈資料

In [2]:
lines = sc.textFile("hdfs://master/user/spark/spark101/avg_temperature/data")


### 這邊 lines 為RDD 物件

In [3]:
lines

hdfs://master/user/spark/spark101/avg_temperature/data MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

## STEP2. 
   **從上面已知 lines 為RDD物件** </br>
   **要打印出來方法有兩種:** </br>
    **1. RDD.collect()** </br>
    **2. RDD.take(<number>)** 
    

In [4]:
tmp = lines.take(10)
tmp


['03011,20070101,0050,AO2 ,-,CLR                                          ,10SM   ,-,30,1,23,29  , 5   ,150,-,0  ,30.14,-,-,AA,-',
 '03011,20070101,0150,AO2 ,-,CLR                                          ,10SM   ,-,28,1,21,31  , 6   ,160,-,0  ,30.13,-,-,AA,-',
 '03011,20070101,0250,AO2 ,-,SCT120                                       ,10SM   ,-,30,1,23,29  , 4   ,150,-,0  ,30.13,-,-,AA,-',
 '03011,20070101,0350,AO2 ,-,CLR                                          ,10SM   ,-,30,1,23,29  , 3   ,140,-,0  ,30.12,-,-,AA,-',
 '03011,20070101,0450,AO2 ,-,CLR                                          ,10SM   ,-,30,5,23,34  , 5   ,150,-,0  ,30.10,-,-,AA,-',
 '03011,20070101,0550,AO2 ,-,CLR                                          ,10SM   ,-,32,1,25,26  , 4   ,130,-,0  ,30.10,-,-,AA,-',
 '03011,20070101,0650,AO2 ,-,CLR                                          ,10SM   ,-,32,1,25,26  , 0   ,000,-,0  ,30.11,-,-,AA,-',
 '03011,20070101,0750,AO2 ,-,CLR                                          ,10SM   ,

## 這邊 is_good function 的用意是過濾掉 第10個element 如果不是為整數的情況

In [5]:
def is_good(record):
    try:
        temp = int(record.split(",")[10])
        print(temp)
    except ValueError:
        return False
    return True

## 以下面為例, 第10個的值就是 30 所以經過 is_good 會回傳True

In [6]:
tmp[0].split(',')

['03011',
 '20070101',
 '0050',
 'AO2 ',
 '-',
 'CLR                                          ',
 '10SM   ',
 '-',
 '30',
 '1',
 '23',
 '29  ',
 ' 5   ',
 '150',
 '-',
 '0  ',
 '30.14',
 '-',
 '-',
 'AA',
 '-']

## RDD.filter (<function>) 
    這邊RDD 只會接回傳為True的row

In [7]:
good_records = lines.filter(is_good)

In [8]:
good_records.take(10)[0]

'03011,20070101,0050,AO2 ,-,CLR                                          ,10SM   ,-,30,1,23,29  , 5   ,150,-,0  ,30.14,-,-,AA,-'

## 這裡的 RDD.map 目的為把日期 跟 地10個element 取出

In [9]:
day_temp = good_records.map(lambda x: (x.split(",")[1], int(x.split(",")[10])))

In [10]:
day_temp.take(10)

[('20070101', 23),
 ('20070101', 21),
 ('20070101', 23),
 ('20070101', 23),
 ('20070101', 23),
 ('20070101', 25),
 ('20070101', 25),
 ('20070101', 21),
 ('20070101', 21),
 ('20070101', 25)]

In [11]:
def my_red(x, y):
    return (x[0] + y [0], x[1] + y[1])

result1 = day_temp.mapValues(lambda x: (x, 1))
result2 = result1.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
#result2 = result1.reduceByKey(my_red)
result3 = result2.map(lambda x: (x[0], x[1][0] / x[1][1]))

In [12]:
day_temp.take(10)

[('20070101', 23),
 ('20070101', 21),
 ('20070101', 23),
 ('20070101', 23),
 ('20070101', 23),
 ('20070101', 25),
 ('20070101', 25),
 ('20070101', 21),
 ('20070101', 21),
 ('20070101', 25)]

## 這邊為result1 的結果 把 原本的value 再分割 並且給予數量1 以方便後面計算

### code: day_temp.mapValues(lambda x: (x, 1))
### lambda 的 x  就是傳入的參數 以上面的例子第一個就是 23 (注意這邊為mapValues 只針對values)
### 所以經過 lambda 回傳為 (23,1) 

In [13]:
result1.take(10)

[('20070101', (23, 1)),
 ('20070101', (21, 1)),
 ('20070101', (23, 1)),
 ('20070101', (23, 1)),
 ('20070101', (23, 1)),
 ('20070101', (25, 1)),
 ('20070101', (25, 1)),
 ('20070101', (21, 1)),
 ('20070101', (21, 1)),
 ('20070101', (25, 1))]

## 這邊為result2 的結果

### code: result1.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
### 這邊是 reduceByKey 所以帶入的參數會跟 value 有關
### 這邊就是 20070101 (key) 的資料他會做相加
### lambda 的部分 x 就是當前的 value 
### 以第一個為例 (23, 1) 那 x[0] = 23 , y[0]=0 , x[1]=1, y[1]=0
### 當到第二個 (21, 1) 那 x[0] = 21 , y[0]=23 , x[1]=1, y[1]=0+1
### 當到第三個 (23, 1) 那 x[0] = 21 , y[0]=23 + 21 , x[1]=1, y[1]=1+1
### 當道第四個 (23, 1) 那 x[0] = 21 , y[0]=23 + 21+21 , x[1]=1, y[1]=1+1+1
### 以此類推

In [14]:
result2.take(10)

[('20070101', (5538, 286)),
 ('20070108', (5861, 264)),
 ('20070110', (7123, 261)),
 ('20070111', (7678, 263)),
 ('20070112', (4859, 263)),
 ('20070113', (3630, 264)),
 ('20070115', (1258, 264)),
 ('20070116', (2427, 264)),
 ('20070119', (4186, 261)),
 ('20070122', (3591, 263))]

## 這邊為 result3 結果
### code: result2.map(lambda x: (x[0], x[1][0] / x[1][1]))

### 注意這邊為map
### 這邊lambda 的x
### 以第一個為例 x[0] = '20070101', x[1]= (5538,286) 所以 x[1][0]= 5538, x[1][1]=286

In [15]:
result3.take(10)

[('20070101', 19.363636363636363),
 ('20070108', 22.200757575757574),
 ('20070110', 27.291187739463602),
 ('20070111', 29.193916349809886),
 ('20070112', 18.475285171102662),
 ('20070113', 13.75),
 ('20070115', 4.765151515151516),
 ('20070116', 9.193181818181818),
 ('20070119', 16.038314176245212),
 ('20070122', 13.653992395437262)]

In [16]:
for line in result3.collect():
    print(line)

('20070101', 19.363636363636363)
('20070108', 22.200757575757574)
('20070110', 27.291187739463602)
('20070111', 29.193916349809886)
('20070112', 18.475285171102662)
('20070113', 13.75)
('20070115', 4.765151515151516)
('20070116', 9.193181818181818)
('20070119', 16.038314176245212)
('20070122', 13.653992395437262)
('20070125', 23.827450980392157)
('20070128', 19.901515151515152)
('20070131', 21.711297071129707)
('20070121', 19.545454545454547)
('20070123', 18.325757575757574)
('20070124', 22.813688212927758)
('20070126', 24.484251968503937)
('20070127', 20.757575757575758)
('20070129', 19.03030303030303)
('20070130', 18.50190114068441)
('20070102', 19.726235741444867)
('20070103', 25.290076335877863)
('20070104', 29.946969696969695)
('20070105', 25.295454545454547)
('20070106', 16.481060606060606)
('20070107', 16.397727272727273)
('20070109', 22.27734375)
('20070114', 9.549242424242424)
('20070117', 14.395437262357415)
('20070118', 14.393939393939394)
('20070120', 21.40530303030303)
